# Can we monitor what7 redditors say recently?

https://github.com/pushshift/api/blob/master/README.md

https://github.com/dmarx/psaw

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
from pmaw import PushshiftAPI
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display

plot_output = widgets.Output()

def display_kw_freq(kw, subreddit, obs_hrs = 24):

    plot_output.clear_output()

    curr_obs_end = dt.timestamp(dt.now())#.replace(minute=0, second=0, microsecond=0))
    curr_obs_start = curr_obs_end - (obs_hrs * 3600)
    prev_obs_end = curr_obs_end - 86400
    prev_obs_start = curr_obs_start - 86400

    api = PushshiftAPI()
    curr_gen = api.search_comments(q=kw, subreddit=subreddit, 
                                   after=int(curr_obs_start), 
                                   before=int(curr_obs_end))
    prev_gen = api.search_comments(q=kw, subreddit=subreddit, 
                                   after=int(prev_obs_start), 
                                   before=int(prev_obs_end))
    curr_df = pd.DataFrame([result for result in curr_gen])
    prev_df = pd.DataFrame([result for result in prev_gen])

    curr_df['set'] = 'Today'
    prev_df['set'] = 'Yesterday'
    prev_df['created_utc'] = df['created_utc'] + 86400
    df = pd.concat([curr_df, prev_df], ignore_index=False)

    df['created_dt'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)
    df['created_dt'] = df['created_dt'].dt.tz_convert('Asia/Hong_Kong')

    hist = px.histogram(df, x='created_dt', color='set', barmode="overlay", #nbins=obs_hrs,
                        title=f"Occurrence of {kw} in r/{subreddit} in the last {obs_hrs} hours")
    with plot_output:
        hist.show()

keyword_text = widgets.Text(
    value='GME',
    placeholder='Type your keywords here',
    description='Keyword: '
)

subreddits = ['wallstreetbets', 'investing', 'finance']
subreddits_dropdown = widgets.Dropdown(
    options=subreddits,
    description='Subreddit: '
)

refresh_btn = widgets.Button(
    description='Refresh'
)

control_box = widgets.TwoByTwoLayout(top_left=keyword_text,
                                     bottom_left=subreddits_dropdown,
                                     bottom_right=refresh_btn,
                                     merge=False)
display(control_box)

def refresh_btn_on_click_event(_):
    display_kw_freq(keyword_text.value, subreddits_dropdown.value)

refresh_btn.on_click(refresh_btn_on_click_event)
#def keyword_text_eventhandler(change):
#    display_kw_freq(change.new, subreddits_dropdown.value)
#def subreddits_dropdown_eventhandler(change):
#    display_kw_freq(keyword_text.value, change.new)
#    print(change.new)

#keyword_text.observe(keyword_text_eventhandler, names='value')
#subreddits_dropdown.observe(subreddits_dropdown_eventhandler, names='value')

In [ ]:
display(plot_output)